### Day 2: Matrix Implementation from Scratch
**Goal: Understand Matrices as Transformations**

In [ ]:
import math
from typing import List, Union
from vector import Vector

class Matrix:
    """
    Matrix Class representing linear transformations.

    A matrix is viewed geometrically as a transformation that maps vectors 
    to new vectors. Each column represents where the basis vectors land
    after the transformation.

    Attributes:
        data: 2D list of numbers representing the matrix
        rows: number of rows in the matrix
        cols: number of columns in the matrix
    Example:
        >>> M = Matrix([[2,0], [0,3]]) # Scaling transformation
        >>> v = Vector([1,1])
        >>> result = M.multiply_vector(v) # should give Vector([2,3])
    """
    def __init__(self, data: List[List[float]]):
        """
        Initialize matrix from 2D list.

        Args:
            data: 2D list where each inner list is a row
                ex: [[1,2], [3,4]] represents a 2x2 matrix.
        """
        self.data = data
        self.rows = len(data) #counts number of inner lists (rows)
        self.cols = len(data[0]) if data else 0 # counts how many elements are in first row (columns). checks for empty matrix 

    def __repr__(self) -> str:
        """
        Return string representation for debugging.

        Returns:
            String like "Matrix([[1,2], [3,4])"
        """
        return f"Matrix({self.data})"
    
    def __str__(self) -> str:
        """

        Pretty print the matrix in reaadable format.

        Returns:
         Multi-line string with formatted matrix
         Example:
             [1.00, 2.00]
             [3.00, 4.00]
        """
        rowstrings = []
        for row in self.data:
            rstring = ", ".join(f"{num:.2f}" for num in row)
            rowstrings.append(rstring)

        return "\n".join(rowstrings)

    def get_column(self, col_index: int) -> Vector:
        """
        Extract a column as a vector.

        Geometric: The i-th column shows where the i-th basis vector lands
        after this transformation.

        Args:
            col_index: Which column to extract (0-indexed)
        Returns:
            Vector containing the column values
        """
        column = [row[col_index] for row in self.data]
        return column

    def multiply_vector(self, vector: Vector) -> Vector:
        """
        Apply this transformation to a vector (matrix-vector multiplication)

        Geometric: Where does this vector land after the transformation 
        represented by this matrix?

        Math: Each component of result is a dot product of a matrix row
        with the input vector.

        Args:
            vector: the vector to transform
        
        Returns:
            Transformed Vector
        
        Raises: ValueError: If matrix columns don't match vector dimension

        Example:
            >>>M = Matrix([[2,0], [0,3]) #scale x by 2, y by 3)
            >>>v = Vector([1,1]) # returns Vector([2,3])
        """
        if (len(vector.components) != self.cols):
            raise ValueError("Dimensions don't match columns")
        result = [vector.dot(Vector(row)) for row in self.data]
        return Vector(result)

    @staticmethod
    def rotation(angle_degrees: float) -> 'Matrix':
        """
        Create a 2D rotation matrix (counterclockwise)
        Rotates vectors counterclockwise by the given angle.

        Args:
            angle_degrees: Rotation angle in degrees
        
        Returns:
            2x2 rotation matrix
        
        Formula:
            [cos(θ)  -sin(θ)]
            [sin(θ)   cos(θ)]
        Example:
            >>> R = Matrix.rotation(90) # 90 degree rotation
            >>> v = Vector([1,0]) # Unit vector along x-axis
            >>> R.multiply_vector(v) # Should give approximately [0,1]
        """
        radians = math.radians(angle_degrees)
        r1 = [math.cos(radians), -math.sin(radians)]
        r2 = [math.sin(radians), math.cos(radians)]

        return Matrix([r1,r2])

    @staticmethod
    def scaling(sx: float, sy:float) -> Matrix:
        """
        Create a 2D scaling matrix.

        Scales x-coords by sx and y-coords by sy.

        Args:
            sx: Scale factor for x-direction
            sy: Scale factor for y-direction

        Returns:
            2x2 scaling matrix

        Mathematical formula:
            [sx 0]
            [0 sy]
        Example:
            >>> S = Matrix.scaling(2, 3)
            >>> v = Vector([1, 1])
            >>> S.multiply_vector(v)  # Returns Vector([2, 3])
        """
        return Matrix([[sx, 0], [0, sy]])

    @staticmethod
    def shear(shear_factor: float) -> Matrix:
        """
        Create a 2D shear matrix.

        Shears the plane along the x-axis. Points move horizontally
        in proportion to their y-coordinate.

        Args:
            shear_factor: how much to shear

        Returns:
            2x2 shear matrix

        Mathematical formula:
            [1  shear_factor]
            [0       1      ]

        Example:
            >>> Sh = Matrix.shear(1)
            >>> v = Vector([0, 1])  # Point at (0,1)
            >>> Sh.multiply_vector(v)  # Returns Vector([1, 1])
        """
        return Matrix([[1, shear_factor], [0, 1]])
        
        
            
        

In [13]:
# Test Cases

M = Matrix([[2, 0], [0, 3]])
v = Vector([3, 4])
result = M.multiply_vector(v)
print(result)
# Expected: Vector([6, 12]) - scaled by 2x and 3y

R = Matrix.rotation(90)
v = Vector([1, 0])
rotated = R.multiply_vector(v)
print(rotated)
# Expected: approximately Vector([0, 1])

v = Vector([3, 4])
scaled = Matrix.scaling(2, 0.5).multiply_vector(v)
print(scaled)
# Expected: Vector([6, 2])

Vector([6, 12])
Vector([6.123233995736766e-17, 1.0])
Vector([6, 2.0])
